# MathematicalProgram에서 비용과 제약조건 업데이트하기(Updating costs and constraints in MathematicalProgram)
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.

최적화 문제를 해결하고 나서 제약 조건과 비용을 약간 조정하여 업데이트된 문제를 다시 해결하고 싶은 경우가 자주 있다. 이러한 유형의 흔한 예시는 모델 예측 제어(MPC)이다. MPC에서는 각 시간 인스턴스에서 새로운 최적화 문제를 해결하는데, 이 문제의 제약 조건/비용은 이전 시간 인스턴스의 것과 약간만 다르다.

새로운 `MathematicalProgram` 객체를 만드는 대신 이전 `MathematicalProgram` 객체에서 제약 조건/비용을 업데이트하고 업데이트된 문제를 해결할 수 있다. 이를 위해 많은 제약 조건/비용 객체는 "update" 함수를 제공하고 있다. 이 튜토리얼에서는 특정 유형의 제약 조건/비용을 업데이트하는 방법을 보여준다.

## `LinearCost` 업데이트하기(Updating `LinearCost`)
선형 비용 $a^Tx + b$의 경우, 선형 계수 `a` 벡터나 상수항 `b`를 업데이트하기 위해 [`LinearCost.UpdateCoefficients()`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.LinearCost.UpdateCoefficients) 함수를 호출할 수 있다.

In [ ]:
from pydrake.solvers import MathematicalProgram, Solve
import numpy as np

prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
cost1 = prog.AddLinearCost(2*x[0] + 3 * x[1] + 2)
print(f"original cost: {cost1}")
prog.AddBoundingBoxConstraint(-1, 1, x)
result = Solve(prog)
print(f"optimal solution: {result.GetSolution(x)}")
print(f"original optimal cost: {result.get_optimal_cost()}")

# Now update the cost to 3x[0] - 4x[1] + 5
cost1.evaluator().UpdateCoefficients(new_a=[3, -4], new_b=5)
print(f"updated cost: {cost1}")
# Solve the optimization problem again with the updated cost.
result = Solve(prog)
print(f"updated optimal solution: {result.GetSolution(x)}")
print(f"updated optimal cost: {result.get_optimal_cost()}")

## `QuadraticCost` 업데이트하기(Updating `QuadraticCost`)
2차 비용 $0.5x^TQx + b'x + c$의 경우, [`QuadraticCost.UpdateCoefficients`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.QuadraticCost.UpdateCoefficients) 함수를 호출하여 그의 $Q, b, c$ 항을 업데이트할 수 있다. 

In [ ]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
cost1 = prog.AddQuadraticCost(x[0]**2 + 2 * x[1]**2 + x[0]*x[1] + 3*x[0] + 5)
print(f" original cost: {cost1}")
cost1.evaluator().UpdateCoefficients(new_Q=[[1., 2], [2., 4]], new_b=[1, 2.], new_c= 4)
print(f" updated cost: {cost1}")

## 제약에 대한 바운드 업데이트하기(Updating the bounds for any constraint)
제약 조건 $lower \le f(x) \le upper$에 대해, 다음 함수를 사용하여 제약 조건의 경계를 업데이트할 수 있습니다:
- [`Constraint.UpdateLowerBound(new_lb)`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.LinearConstraint.UpdateLowerBound) : lower bound를 `new_lb`로 변경
- [`Constraint.UpdateUpperBound(new_ub)`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.LinearConstraint.UpdateUpperBound) : upper bound를 `new_ub`로 변경
- [`Constraint.set_bounds(new_lb, new_ub)`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.LinearConstraint.set_bounds) lower와 upper bounds 모두 변경

In [ ]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
constraint1 = prog.AddLinearConstraint(x[0] + 3 * x[1] <= 2)
print(f"original constraint: {constraint1}")
constraint1.evaluator().UpdateLowerBound([-1])
print(f"updated constraint: {constraint1}")
constraint1.evaluator().UpdateUpperBound([3])
print(f"updated constraint: {constraint1}")
constraint1.evaluator().set_bounds(new_lb=[-5], new_ub=[10])
print(f"updated constraint: {constraint1}")

## 선형 제약 계수와 바운드 업데이트(Update linear constraint coefficients and bounds)
선형 제약 조건 $lower \le Ax \le upper$에 대해 [`LinearConstraint.UpdateCoefficients(new_A, new_lb, new_ub)`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.LinearConstraint.UpdateCoefficients) 함수를 호출하여 제약 조건을 $new_{lb} \le new_A* x\le new_ub$로 업데이트할 수 있다.

선형 등식 제약 조건 $Ax = b$에 대해 [`LinearEqualityConstraint.UpdateCoefficients(Aeq, beq)`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.LinearEqualityConstraint.UpdateCoefficients) 함수를 호출하여 제약 조건을 $\text{Aeq}*x=\text{beq}$로 업데이트할 수 있다.

In [ ]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
linear_constraint = prog.AddLinearConstraint(3 * x[0] + 4 * x[1] <= 5)
linear_eq_constraint = prog.AddLinearConstraint(5 * x[0] + 2 * x[1] == 3)
print(f"original linear constraint: {linear_constraint}")
linear_constraint.evaluator().UpdateCoefficients(new_A = [[1, 3]], new_lb=[-2], new_ub=[3])
print(f"updated linear constraint: {linear_constraint}")

print(f"original linear equality constraint: {linear_eq_constraint}")
linear_eq_constraint.evaluator().UpdateCoefficients(Aeq=[[3, 4]], beq=[2])
print(f"updated linear equality constraint: {linear_eq_constraint}")